In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install mediapipe
!pip install --upgrade scikit-learn

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
DATA_PATH = os.path.join('/kaggle/input/mpdata/MP_DATA')

actions = np.array(['a','b','c','d','e','f','g',
                    'h','i','j','k','l','m','n',
                    'o','p','q','r','s','t','u',
                    'v','w','x','y','z'])

#no of videos
no_sequnces = 30

#number of frames
sequnce_length = 30 


In [ ]:
DATA_PATH = os.path.join('/kaggle/input/wlpsl/WLPSL')
action = []
for subdir,dirs,files in os.walk(DATA_PATH +"/Videos"):
    for dirc in dirs:
        action.append(dirc)

actions = np.array(action)

#no of videos
no_sequnces = 30

#number of frames
sequnce_length = 30 


In [ ]:
actions

In [ ]:
len(actions)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num,label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences,labels = [],[]
for action in actions:
    for sequence in range(no_sequnces):
        window = []
        for frame_num in range(sequnce_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.unique(labels)

In [ ]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.05)

In [ ]:
print(x.shape)   # should be (num_samples, 30, 1662) or similar
print(y.shape)   # should be (num_samples,)
print(np.unique(np.argmax(y, axis=1)))  # should show 26 unique numbers (0–25)


In [ ]:
import joblib

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_train = X_train_scaled.reshape(X_train.shape)


In [ ]:

X_test_reshaped = x_test.reshape(-1, x_test.shape[-1])
X_test_scaled = scaler.transform(X_test_reshaped)
x_test = X_test_scaled.reshape(x_test.shape)


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(700)
X_train_reduced = pca.fit_transform(X_train.reshape(-1, X_train.shape[-1]))
X_train_reduced = X_train_reduced.reshape(X_train.shape[0], X_train.shape[1], 700)


In [ ]:
X_test_reduced = pca.transform(x_test.reshape(-1, x_test.shape[-1]))
X_test_reduced = X_test_reduced.reshape(x_test.shape[0], x_test.shape[1], 700)


In [ ]:
import joblib
joblib.dump(scaler,"scalerV2.pkl")
joblib.dump(pca,"pcaV2_700.pkl")


In [ ]:
import numpy as np
import random

def augment_sequence(sequence, noise_std=0.001, dropout_prob=0.1):
    """
    sequence: np.array of shape (30, 1662)
    """
    seq = sequence.copy()

    # 1. Add small Gaussian noise
    seq += np.random.normal(0, noise_std, seq.shape)

    # 2. Random feature scaling
    seq *= np.random.uniform(0.95, 1.02)

    # 3. Random frame dropout
    if random.random() < dropout_prob:
        drop_idx = np.random.choice(seq.shape[0], size=1, replace=False)
        seq = np.delete(seq, drop_idx, axis=0)
        # Pad back to 30 frames (duplicate last frame)
        seq = np.pad(seq, ((0, 1), (0, 0)), mode='edge')

    return seq


In [ ]:
X_augmented = []
y_augmented = []

for x, y in zip(X_train_reduced, y_train):
    X_augmented.append(x)
    y_augmented.append(y)

    # Add one augmented version of each sequence
    X_augmented.append(augment_sequence(x))
    y_augmented.append(y)

X_augmented = np.array(X_augmented)
y_augmented = np.array(y_augmented)


In [ ]:
X_augmented.shape

In [ ]:
y_augmented.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout,BatchNormalization,Bidirectional
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2


log_dir = os.path.join('logs')

tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=40,              # stop if val_loss doesn't improve for 20 epochs
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=15,
    min_lr=1e-6,
    verbose=1
)


In [ ]:
X_train_reduced.shape

In [ ]:
model = Sequential()

model.add(Bidirectional(LSTM(64,return_sequences=True,activation="tanh",input_shape=(30,700))))
model.add(BatchNormalization())
model.add(Dropout(0.2))



model.add(Bidirectional(LSTM(64,return_sequences=False,activation="tanh")))
model.add(Dropout(0.2))


model.add(Dense(128,activation='relu',kernel_regularizer=l2(5e-6)))
model.add(Dropout(0.15))

model.add(Dense(64,activation='relu',))
model.add(Dropout(0.15))

model.add(Dense(32,activation='relu',))
model.add(Dropout(0.1))

model.add(Dense(actions.shape[0],activation='softmax'))





In [ ]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=['accuracy'])


In [ ]:
model.fit(X_augmented,y_augmented,validation_split=0.2,epochs=2000,callbacks=[tb_callback,early_stop,checkpoint,reduce_lr])

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
loss,acc = model.evaluate(X_test_reduced,y_test)

In [ ]:
print(acc)

In [ ]:
model.save("model_12.keras")

In [ ]:
model = load_model("model_11.keras")

# Base CNN Model

In [ ]:
import cv2,os

In [ ]:
DATA_PATH = os.path.join('/kaggle/input/asl-dataset/asl_dataset/asl_dataset')
output_path = "/kaggle/working/outputs"
FRAMES_PER_VIDEO = 30
FRAME_SIZE = (224,224)
OUTPUT_PATH = "/kaggle/working/outputs"

In [ ]:
os.listdir(DATA_PATH)

In [ ]:
import cv2, os

def extract_frames(video_path, output_folder, frame_count=60):  
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames == 0:
        print(f"Skipping (no frames): {video_path}")
        return 
    
    # FIXED evenly spaced frames
    frame_indices = [int(i * total_frames / frame_count) for i in range(frame_count)]
    frame_indices = sorted(set(frame_indices))

    saved = 0
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if count in frame_indices:
            frame = cv2.resize(frame, FRAME_SIZE)
            frame_name = f"frame_{saved}.jpg"
            cv2.imwrite(os.path.join(output_folder, frame_name), frame)
            saved += 1
        
        count += 1

        if saved >= len(frame_indices):
            break

    cap.release()
    print(f"Extracted {saved} frames from {video_path}")


In [ ]:
def extract_all_frames():
    for class_name in os.listdir(DATA_PATH):
        class_path = os.path.join(DATA_PATH, class_name)

        if not os.path.isdir(class_path):
            continue

        print(f"\nProcessing Class: {class_name}")

        output_dir = os.path.join(output_path, class_name)
        os.makedirs(output_dir, exist_ok=True)

        for video_file in os.listdir(class_path):
            if video_file.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):

                video_path = os.path.join(class_path, video_file)

                video_output_folder = os.path.join(
                    output_dir,
                    video_file.split('.')[0]
                )
                os.makedirs(video_output_folder, exist_ok=True)

                print(f"Extracting ALL frames from: {video_file}")
                extract_frames(video_path, video_output_folder)

    print("\nAll Frames Extracted Successfully")


In [ ]:
extract_all_frames()

In [ ]:
def preprocess_frame(frame_path,dataset_dir,frame):
    os.makedirs(dataset_dir,exist_ok=True)
    
    img = cv2.imread(frame_path)
    resized_img = cv2.resize(img,(224,224),interpolation=cv2.INTER_AREA)
    rgb_img = cv2.cvtColor(resized_img,cv2.COLOR_BGR2RGB)
    
    norm_img = cv2.normalize(
    rgb_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
     
    final_img = norm_img
    bgr_final = cv2.cvtColor(final_img, cv2.COLOR_RGB2BGR)
    folder_path = os.path.join(dataset_dir)
    cv2.imwrite(os.path.join(folder_path,frame),bgr_final)

    


In [ ]:
preprocess_output_path = "/kaggle/working/new_dataset_asl"
alphabetinput = '/kaggle/input/dataset-pakistani-sign-language/dataset'
os.makedirs(preprocess_output_path,exist_ok=True)

def process_all_frames():
    for class_name in os.listdir(DATA_PATH):
        class_path = os.path.join(DATA_PATH,class_name)
        
        if not os.path.isdir(class_path):
            continue
    
        for video_num in os.listdir(class_path):
            frame_dir = os.path.join(class_path,video_num)
    
            for frame in os.listdir(frame_dir):
                
                frame_path = os.path.join(frame_dir,frame)
                
                dataset_path = os.path.join(preprocess_output_path,class_name)
                dataset_dir = os.path.join(dataset_path,video_num)
                print(dataset_dir)
                print(f"Preprocessing {frame} of {class_name}")
                preprocess_frame(frame_path,dataset_dir,frame)
                
                
    


In [ ]:
#for alphabets
def process_all_frames():
    for class_name in os.listdir(DATA_PATH):
        class_path = os.path.join(DATA_PATH, class_name)

        if not os.path.isdir(class_path):
            continue

        # output folder for this class
        output_class_dir = os.path.join(preprocess_output_path, class_name)
        os.makedirs(output_class_dir, exist_ok=True)

        # iterate all images
        for frame in os.listdir(class_path):
            frame_path = os.path.join(class_path, frame)

            if not frame.lower().endswith((".jpg", ".jpeg", ".png")):
                continue  # skip non-images
            
            print(f"Preprocessing {frame} of {class_name}")
            preprocess_frame(frame_path, output_class_dir, frame)


In [ ]:
process_all_frames()


In [ ]:
import tensorflow as tf

In [ ]:
DATASET_PATH = "/kaggle/working/new_dataset_asl"

In [ ]:
import os
os.listdir(DATASET_PATH)

In [ ]:
batch_size = 32
img_size= (224,224)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)


In [ ]:
BIGGER_ASL_DATASET = "/kaggle/input/american-sign-language/ASL_Dataset"

In [ ]:
train = "/kaggle/input/asl-dataset-mine/archive/ASL_Alphabet_Dataset/asl_alphabet_train"
test = "/kaggle/input/asl-dataset-mine/archive/ASL_Alphabet_Dataset/asl_alphabet_test"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
import tensorflow as tf
batch_size = 32
img_size= (224,224)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test,
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)


In [ ]:
class_names = train_ds.class_names
print(class_names)

for images,labels in train_ds.take(1):
    print(images.shape)
    print(labels.shape)

In [ ]:
len(class_names)

In [ ]:
data_augmentation=tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.15),
    tf.keras.layers.RandomTranslation(0.1,0.1),
    tf.keras.layers.RandomZoom(0.15),
    # tf.keras.layers.RandomBrightness(0.2),
])
normalization_layer = tf.keras.layers.Rescaling(1./255)

brightness_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomBrightness(0.2,value_range=(0.0,1.0)),
])

In [ ]:
def prepare_dataset(ds,augment=False):
    if augment:
        ds = ds.map(lambda x,y:(data_augmentation(x,training=True),y),num_parallel_calls=tf.data.AUTOTUNE)
        
    ds = ds.map(lambda x,y:(normalization_layer(x),y), num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        ds = ds.map(lambda x, y:(brightness_augmentation(x,training=True),y), num_parallel_calls=tf.data.AUTOTUNE)
        # ds = ds.map(lambda x, y:(tf.clip_by_values(x,0.0,1.0),y), num_parallel_calls=tf.data.AUTOTUNE)
        
        

    

    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds
    

train_ds = prepare_dataset(train_ds,augment=True)
val_ds = prepare_dataset(val_ds,augment=False)
test_ds = prepare_dataset(test_ds,augment=True)



for images, labels in train_ds.take(1):
    print(f"After normalization - Min: {images.numpy().min():.3f}, Max: {images.numpy().max():.3f}, Mean: {images.numpy().mean():.3f}")
    # Should now print: Min: 0.0, Max: 1.0 (approximately)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,Rescaling



In [ ]:
num_classes=len(class_names)

model = Sequential([
    
    Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    Dropout(0.25),
    
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.3),

    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.3),

    Flatten(),

    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(64,activation='relu'),
    Dropout(0.5),
    Dense(num_classes,activation='softmax')
    
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(
    monitor='val_loss',           # Monitor validation loss
    patience=3,                    # Stop if no improvement for 5 epochs
    restore_best_weights=True,     # Restore weights from best epoch
    verbose=1                      # Print messages
)

# Optional: Save the best model during training
model_checkpoint = ModelCheckpoint(
    'best_model.keras',           # or 'best_model.h5'
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=35,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint]

)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
ax[0].plot(history.history['accuracy'], label='Train Acc')
ax[0].plot(history.history['val_accuracy'], label='Val Acc')
ax[0].set_title("Accuracy")
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Accuracy")
ax[0].legend()
ax[0].grid(True)

# Loss
ax[1].plot(history.history['loss'], label='Train Loss')
ax[1].plot(history.history['val_loss'], label='Val Loss')
ax[1].set_title("Loss")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Loss")
ax[1].legend()
ax[1].grid(True)

plt.show()
plt.savefig("signbuddy_cnn.png",dpi=300,bbox_inches='tight')

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_true = []
y_pred = []

for batch_x, batch_y in test_ds:
    # predict on this batch
    batch_preds = model.predict(batch_x, verbose=0)
    batch_pred_classes = np.argmax(batch_preds, axis=1)

    # store
    y_true.extend(batch_y.numpy())
    y_pred.extend(batch_pred_classes)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_true, y_pred)
print(cm)

test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Get one batch for visualization
for images, labels in test_ds.take(1):
    preds = model.predict(images)
    pred_classes = np.argmax(preds, axis=1)
    
    plt.figure(figsize=(15, 10))
    for i in range(min(9, len(images))):
        plt.subplot(3, 3, i+1)
        plt.imshow(images[i].numpy())
        plt.title(f"True: {class_names[labels[i]]}\nPred: {class_names[pred_classes[i]]}")
        plt.axis('off')
    plt.tight_layout()

    plt.show()

In [ ]:
model.save("signbuddy_cnn_asl_model_6.h5")
model.save("signbuddy_cnn_asl_model_6.keras")

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('/kaggle/working/signbuddy_cnn_asl_model_4.keras')

In [ ]:
# Evaluate on the SAME test set that gave you 100% accuracy
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc}")

# Get predictions for the entire test set
predictions = []
true_labels = []

for images, labels in test_ds:
    preds = model.predict(images, verbose=0)
    predictions.extend(preds.argmax(axis=1))
    true_labels.extend(labels.numpy())

from sklearn.metrics import classification_report
print(classification_report(true_labels, predictions, target_names=class_names))

In [ ]:
import tensorflow as tf



converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("asl_alphabert_model.tflite", "wb") as f:
    f.write(tflite_model)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, 
    BatchNormalization, GlobalAveragePooling2D
)
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
)
import numpy as np

# ============================================================================
# LOAD DATA WITH AUGMENTATION + PREFETCHING
# ============================================================================

# TRAIN_PATH = "/kaggle/input/american-sign-language/ASL_Dataset/Train"
# TEST_PATH = "/kaggle/input/american-sign-language/ASL_Dataset/Test"

batch_size = 32  # Reduced from 32 - uses half the memory!
img_size = (224, 224)

# Load datasets using tf.keras.utils
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH,
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")
print(f"Classes: {class_names}")

# ============================================================================
# DATA AUGMENTATION LAYER - FIXED VERSION
# ============================================================================

# Option 1: Augment on raw images, then normalize (SAFEST)
data_augmentation_raw = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.15),
    tf.keras.layers.RandomTranslation(0.1, 0.1, fill_mode='nearest'),
    tf.keras.layers.RandomZoom(0.15, fill_mode='nearest'),
    # Skip RandomBrightness here - will add after normalization
])

# Normalization layer
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Brightness adjustment (works better on normalized data)
brightness_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomBrightness(0.2, value_range=(0.0, 1.0)),  # Specify range!
])

# Apply augmentation in correct sequence
def prepare_dataset(ds, augment=False):
    if augment:
        # Step 1: Geometric augmentations on raw images (0-255)
        ds = ds.map(lambda x, y: (data_augmentation_raw(x, training=True), y),
                    num_parallel_calls=tf.data.AUTOTUNE)
    
    # Step 2: Normalize to 0-1
    ds = ds.map(lambda x, y: (normalization_layer(x), y), 
                num_parallel_calls=tf.data.AUTOTUNE)
    
    if augment:
        # Step 3: Brightness on normalized images (0-1)
        ds = ds.map(lambda x, y: (brightness_augmentation(x, training=True), y),
                    num_parallel_calls=tf.data.AUTOTUNE)
    
    # Step 4: Cache and prefetch
    # REMOVED CACHE to save GPU memory
    # if not augment:
    #     ds = ds.cache()  # Only cache val/test
    
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return ds

train_ds = prepare_dataset(train_ds, augment=True)
val_ds = prepare_dataset(val_ds, augment=False)
test_ds = prepare_dataset(test_ds, augment=False)

# Verify normalization is working correctly
print("\n🔍 Verifying normalization on TRAINING data (with augmentation)...")
for images, labels in train_ds.take(1):
    min_val = tf.reduce_min(images).numpy()
    max_val = tf.reduce_max(images).numpy()
    mean_val = tf.reduce_mean(images).numpy()
    print(f"  Min:  {min_val:.3f}")
    print(f"  Max:  {max_val:.3f}")
    print(f"  Mean: {mean_val:.3f}")
    if min_val >= 0.0 and max_val <= 1.0:
        print("  ✅ CORRECT: Values in [0, 1] range!")
    else:
        print("  ❌ ERROR: Values outside [0, 1] range!")

print("\n🔍 Verifying normalization on VALIDATION data (no augmentation)...")
for images, labels in val_ds.take(1):
    min_val = tf.reduce_min(images).numpy()
    max_val = tf.reduce_max(images).numpy()
    mean_val = tf.reduce_mean(images).numpy()
    print(f"  Min:  {min_val:.3f}")
    print(f"  Max:  {max_val:.3f}")
    print(f"  Mean: {mean_val:.3f}")
    if min_val >= 0.0 and max_val <= 1.0:
        print("  ✅ CORRECT: Values in [0, 1] range!")
    else:
        print("  ❌ ERROR: Values outside [0, 1] range!")
    
print("\n✅ Datasets prepared with caching and prefetching!")

# ============================================================================
# IMPROVED CNN MODEL
# ============================================================================

def create_improved_model(num_classes):
    model = Sequential([
        # Block 1
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224,224,3)),
        BatchNormalization(),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        # Block 2
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        # Block 3
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.3),
        
        # Block 4
        Conv2D(256, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.3),
        
        # Classification head
        GlobalAveragePooling2D(),  # Better than Flatten
        
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

model = create_improved_model(num_classes)

# ============================================================================
# ADVANCED CALLBACKS
# ============================================================================

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    'best_asl_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

# ============================================================================
# COMPILE WITH OPTIMIZED SETTINGS
# ============================================================================

# CRITICAL: Match loss function to label format!
# sparse_categorical_crossentropy for integer labels (0, 1, 2, ...)
# categorical_crossentropy for one-hot labels ([0,0,1,0,...])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',  # For integer labels
    metrics=['accuracy']
)

model.summary()

print("\n✅ Model compiled with sparse_categorical_crossentropy")

# ============================================================================
# TRAINING
# ============================================================================

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    verbose=1
)

# ============================================================================
# EVALUATION
# ============================================================================

test_loss, test_accuracy = model.evaluate(test_ds)
print(f"\n{'='*60}")
print(f"FINAL TEST ACCURACY: {test_accuracy*100:.2f}%")
print(f"FINAL TEST LOSS: {test_loss:.4f}")
print(f"{'='*60}")

# ============================================================================
# DETAILED METRICS
# ============================================================================

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Get predictions
y_pred = []
y_true = []

for batch_x, batch_y in test_ds:
    preds = model.predict(batch_x, verbose=0)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(batch_y.numpy())

y_pred = np.array(y_pred)
y_true = np.array(y_true)

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(20, 16))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# TRAINING CURVES
# ============================================================================

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
ax[0].plot(history.history['accuracy'], label='Train Acc', linewidth=2)
ax[0].plot(history.history['val_accuracy'], label='Val Acc', linewidth=2)
ax[0].set_title("Model Accuracy", fontsize=14, fontweight='bold')
ax[0].set_xlabel("Epoch", fontsize=12)
ax[0].set_ylabel("Accuracy", fontsize=12)
ax[0].legend(fontsize=11)
ax[0].grid(True, alpha=0.3)

# Loss
ax[1].plot(history.history['loss'], label='Train Loss', linewidth=2)
ax[1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
ax[1].set_title("Model Loss", fontsize=14, fontweight='bold')
ax[1].set_xlabel("Epoch", fontsize=12)
ax[1].set_ylabel("Loss", fontsize=12)
ax[1].legend(fontsize=11)
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("training_curves.png", dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# SAVE MODEL
# ============================================================================

model.save("asl_final_model.keras")
print("\nModel saved as 'asl_final_model.keras'")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("asl_model_1.tflite", "wb") as f:
    f.write(tflite_model)
    
print("TFLite model saved as 'asl_model0.tflite'")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, 
    BatchNormalization, GlobalAveragePooling2D, Input
)
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
)
from tensorflow.keras.regularizers import l2
import numpy as np

# ============================================================================
# MEMORY OPTIMIZATION SETTINGS
# ============================================================================

# Clear any previous models from memory
tf.keras.backend.clear_session()

# Enable memory growth for GPU (prevents allocation of all GPU memory at once)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU memory growth enabled for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(f"⚠️  {e}")

# Set TensorFlow to use less memory
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

print("\n" + "="*60)
print("MEMORY-EFFICIENT CONFIGURATION")
print("="*60)

# ============================================================================
# ANTI-OVERFITTING CONFIGURATION
# ============================================================================

TRAIN_PATH = "/kaggle/input/american-sign-language/ASL_Dataset/Train"
TEST_PATH = "/kaggle/input/american-sign-language/ASL_Dataset/Test"

batch_size = 32  # REDUCED from 32 - uses half the memory!
img_size = (128, 128)  # REDUCED from 128 - 30% less memory per image!

# ============================================================================
# AGGRESSIVE DATA AUGMENTATION (Critical for Generalization)
# ============================================================================

data_augmentation = tf.keras.Sequential([
    # Geometric transformations
    tf.keras.layers.RandomRotation(0.2),  # ±20% rotation
    tf.keras.layers.RandomTranslation(0.15, 0.15),  # Shift hands
    tf.keras.layers.RandomZoom(0.2),  # Zoom in/out
    tf.keras.layers.RandomFlip("horizontal"),  # Mirror hands
    
    # Color/lighting variations (critical for generalization!)
    tf.keras.layers.RandomBrightness(0.3),  # Lighting changes
    tf.keras.layers.RandomContrast(0.3),  # Contrast variations
    
    # Optional: Add noise to prevent memorization
    # tf.keras.layers.GaussianNoise(0.01),
])

# normalization = tf.keras.layers.Rescaling(1./255)

# ============================================================================
# LOAD DATA - MEMORY EFFICIENT VERSION
# ============================================================================

# CRITICAL: Don't load all data at once!
# Use generators to load batches on-the-fly

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,
    label_mode='int'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    label_mode='int'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH,
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")
print(f"Classes: {class_names}")

# Apply augmentation - NO CACHING (saves memory!)
def prepare_dataset(ds, augment=False):

    
    # Augment if training
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=tf.data.AUTOTUNE)
    
    # CRITICAL: Only prefetch, NO cache() to save memory
    return ds.prefetch(buffer_size=tf.data.AUTOTUNE)

train_ds = prepare_dataset(train_ds, augment=True)
val_ds = prepare_dataset(val_ds, augment=False)
test_ds = prepare_dataset(test_ds, augment=False)

print("✅ Data loaded WITHOUT caching (memory efficient!)")

# ============================================================================
# SOLUTION 1: TRANSFER LEARNING (BEST FOR OVERFITTING)
# ============================================================================

def create_transfer_learning_model(num_classes, trainable_layers=10):
    """
    MEMORY EFFICIENT: Smaller input + fewer trainable layers
    """
    
    # Load pre-trained model
    base_model = MobileNetV2(
        input_shape=(96, 96, 3),  # Smaller input size
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze MORE layers (less memory during training)
    base_model.trainable = True
    for layer in base_model.layers[:-trainable_layers]:  # Only train last 10 layers
        layer.trainable = False
    
    print(f"✅ MobileNetV2: Only {trainable_layers} layers trainable (memory efficient)")
    
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        
        # SMALLER dense layers
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),  # Reduced from 256
        Dropout(0.5),
        
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),  # Reduced from 128
        Dropout(0.4),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# ============================================================================
# SOLUTION 2: LIGHTWEIGHT CNN WITH HEAVY REGULARIZATION
# ============================================================================

def create_regularized_cnn(num_classes):
    """
    MEMORY EFFICIENT: Smaller, simpler CNN
    """
    
    model = Sequential([
        # Block 1
        Conv2D(32, (3,3), activation='relu', padding='same', 
               kernel_regularizer=l2(0.001), input_shape=(96,96,3)),  # 96x96 input
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.3),
        
        # Block 2 - SMALLER filters
        Conv2D(64, (3,3), activation='relu', padding='same', 
               kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.3),
        
        # Block 3 - Removed 4th block to save memory
        Conv2D(128, (3,3), activation='relu', padding='same', 
               kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.4),
        
        # Smaller classification head
        GlobalAveragePooling2D(),
        
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),  # Reduced from 256
        Dropout(0.5),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# ============================================================================
# SOLUTION 3: TINY MODEL (Harder to Overfit)
# ============================================================================

def create_tiny_model(num_classes):
    """
    ULTRA LIGHTWEIGHT - use if still crashing
    """
    
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(96,96,3)),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        GlobalAveragePooling2D(),
        
        Dense(64, activation='relu'),  # Very small
        Dropout(0.5),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# ============================================================================
# CREATE MODEL (Choose based on memory)
# ============================================================================

# OPTION 1: If still crashing, use TINY model
model = create_tiny_model(num_classes)

# OPTION 2: If tiny works, try regularized CNN
# model = create_regularized_cnn(num_classes)

# OPTION 3: If you have enough memory, use transfer learning (best accuracy)
# model = create_transfer_learning_model(num_classes, trainable_layers=10)

print(f"\n✅ Model created with {model.count_params():,} parameters")

# ============================================================================
# COMPILE WITH LOWER LEARNING RATE (Prevents Overfitting)
# ============================================================================

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Lower LR!
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ============================================================================
# ANTI-OVERFITTING CALLBACKS
# ============================================================================

# Stop early when validation stops improving
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Watch validation, not training
    patience=10,  # Stop if no improvement for 10 epochs
    restore_best_weights=True,
    verbose=1,
    mode='max'
)

# Save best model based on VALIDATION accuracy
model_checkpoint = ModelCheckpoint(
    'best_generalized_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Reduce learning rate when stuck
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

# ============================================================================
# TRAINING WITH VALIDATION MONITORING
# ============================================================================

print("\n" + "="*60)
print("TRAINING WITH ANTI-OVERFITTING STRATEGIES")
print("="*60)

history = model.fit(
    train_ds,
    validation_data=val_ds,  # Monitor validation constantly
    epochs=50,  # Will stop early if overfitting
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    verbose=1
)

# ============================================================================
# DETECT OVERFITTING IN RESULTS
# ============================================================================

print("\n" + "="*60)
print("OVERFITTING CHECK")
print("="*60)

final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
gap = final_train_acc - final_val_acc

print(f"Final Training Accuracy:   {final_train_acc*100:.2f}%")
print(f"Final Validation Accuracy: {final_val_acc*100:.2f}%")
print(f"Gap (Train - Val):         {gap*100:.2f}%")

if gap > 0.15:  # More than 15% gap
    print("\n⚠️  HIGH OVERFITTING DETECTED!")
    print("Solutions:")
    print("  1. Add MORE data augmentation")
    print("  2. Use SMALLER model (fewer parameters)")
    print("  3. Increase Dropout rates")
    print("  4. Get more training data")
elif gap > 0.05:
    print("\n⚠️  Mild overfitting - manageable")
else:
    print("\n✅ Good generalization!")

# ============================================================================
# TEST ON COMPLETELY UNSEEN DATA (MOST IMPORTANT!)
# ============================================================================

print("\n" + "="*60)
print("REAL-WORLD TEST (Different people/conditions)")
print("="*60)

test_loss, test_accuracy = model.evaluate(test_ds)

print(f"\nTraining Accuracy:   {final_train_acc*100:.2f}%")
print(f"Validation Accuracy: {final_val_acc*100:.2f}%")
print(f"TEST Accuracy:       {test_accuracy*100:.2f}%  ← REAL PERFORMANCE")
print(f"{'='*60}")

# THIS IS THE NUMBER THAT MATTERS!
if test_accuracy < 0.7 and final_val_acc > 0.95:
    print("\n🚨 SEVERE DATA LEAKAGE DETECTED!")
    print("Val = {:.0f}% but Test = {:.0f}%".format(final_val_acc*100, test_accuracy*100))
    print("\nYour validation set is TOO SIMILAR to training!")
    print("\nSOLUTIONS:")
    print("  1. Get test images from DIFFERENT people")
    print("  2. Use DIFFERENT backgrounds")
    print("  3. Use DIFFERENT lighting conditions")
    print("  4. Use DIFFERENT cameras/phones")
    print("  5. Add natural variations (jewelry, sleeves, skin tones)")
elif test_accuracy < 0.85:
    print("\n⚠️  Model struggles with real-world variety")
    print("Need MORE diverse training data!")
else:
    print("\n✅ Good generalization to new data!")

# ============================================================================
# ANALYZE WHAT THE MODEL IS ACTUALLY LEARNING
# ============================================================================

print("\n" + "="*60)
print("CHECKING: Is the model learning HAND SHAPES or BACKGROUND?")
print("="*60)

# Sample some test predictions to see what's failing
sample_images = []
sample_labels = []
sample_preds = []

for batch_x, batch_y in test_ds.take(3):
    preds = model.predict(batch_x, verbose=0)
    sample_images.extend(batch_x.numpy()[:5])
    sample_labels.extend(batch_y.numpy()[:5])
    sample_preds.extend(np.argmax(preds[:5], axis=1))

print("\nSample Predictions:")
for i in range(min(15, len(sample_labels))):
    true_label = class_names[sample_labels[i]]
    pred_label = class_names[sample_preds[i]]
    status = "✅" if true_label == pred_label else "❌"
    print(f"  {status} True: {true_label:2s} | Predicted: {pred_label:2s}")

# ============================================================================
# VISUALIZATION: Spot Overfitting
# ============================================================================

import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot - should see train/val stay close together
ax[0].plot(history.history['accuracy'], label='Train Acc', linewidth=2)
ax[0].plot(history.history['val_accuracy'], label='Val Acc', linewidth=2)
ax[0].set_title("Training vs Validation Accuracy", fontsize=14, fontweight='bold')
ax[0].set_xlabel("Epoch", fontsize=12)
ax[0].set_ylabel("Accuracy", fontsize=12)
ax[0].legend(fontsize=11)
ax[0].grid(True, alpha=0.3)

# Add annotation if overfitting
if gap > 0.15:
    ax[0].text(0.5, 0.5, 'OVERFITTING!', 
               transform=ax[0].transAxes, 
               fontsize=20, color='red', alpha=0.3,
               ha='center', va='center')

# Loss plot - validation should decrease smoothly
ax[1].plot(history.history['loss'], label='Train Loss', linewidth=2)
ax[1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
ax[1].set_title("Training vs Validation Loss", fontsize=14, fontweight='bold')
ax[1].set_xlabel("Epoch", fontsize=12)
ax[1].set_ylabel("Loss", fontsize=12)
ax[1].legend(fontsize=11)
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("overfitting_check.png", dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# DETAILED METRICS
# ============================================================================

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

y_pred = []
y_true = []

for batch_x, batch_y in test_ds:
    preds = model.predict(batch_x, verbose=0)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(batch_y.numpy())

y_pred = np.array(y_pred)
y_true = np.array(y_true)

print("\nPer-Class Performance (find weak classes):")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(16, 14))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Generalized Model')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('generalized_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# SAVE BEST MODEL
# ============================================================================

model.save("asl_generalized_model.keras")
print("\n✅ Generalized model saved!")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("asl_generalized.tflite", "wb") as f:
    f.write(tflite_model)
    
print("✅ TFLite model saved!")

# ============================================================================
# ADDITIONAL TIPS FOR 100% VAL BUT BAD REAL-WORLD PERFORMANCE
# ============================================================================

print("\n" + "="*60)
print("WHY 100% VAL BUT FAILS ON NEW IMAGES?")
print("="*60)
print("\n🔍 Most Common Causes:")
print("  1. SAME PERSON in train/val/test")
print("     → Model memorizes YOUR hand, not hand shapes")
print("\n  2. SAME BACKGROUND (white wall, desk, etc)")
print("     → Model uses background to classify, not hand!")
print("\n  3. SAME LIGHTING/CAMERA")
print("     → Model memorizes photo conditions")
print("\n  4. INSUFFICIENT VARIETY")
print("     → Need multiple people, skin tones, angles")
print("\n✅ SOLUTIONS:")
print("  1. Collect images from 5+ different people")
print("  2. Use varied backgrounds (blur them if possible)")
print("  3. Different lighting (indoor, outdoor, bright, dim)")
print("  4. Different hand positions/rotations")
print("  5. Include hand accessories (rings, watches, sleeves)")
print("  6. Use data from multiple cameras/phones")
print("\n💡 QUICK TEST:")
print("  Take 10 photos of YOUR OWN hand right now")
print("  → If model gets <50% accuracy, it's memorizing!")
print("="*60)

# ============================================================================
# EMERGENCY FIX: Background Randomization
# ============================================================================

print("\n" + "="*60)
print("EMERGENCY AUGMENTATION (if can't get new data)")
print("="*60)

# Add this to your data_augmentation
extreme_augmentation = tf.keras.Sequential([
    # Original augmentations
    tf.keras.layers.RandomRotation(0.3),  # Even more rotation
    tf.keras.layers.RandomTranslation(0.2, 0.2),
    tf.keras.layers.RandomZoom(0.3),
    tf.keras.layers.RandomFlip("horizontal"),
    
    # EXTREME lighting changes
    tf.keras.layers.RandomBrightness(0.4),  # Bigger range
    tf.keras.layers.RandomContrast(0.5),
    
    # Color distortion to ignore background colors
    # This forces model to focus on hand shape, not colors
    tf.keras.layers.Lambda(lambda x: tf.image.random_hue(x, 0.2)),
    tf.keras.layers.Lambda(lambda x: tf.image.random_saturation(x, 0.5, 1.5)),
])

print("Added EXTREME augmentation to break background memorization")

In [ ]:
TRAIN_PATH="/kaggle/input/asl-dataset-mine/archive/ASL_Alphabet_Dataset/asl_alphabet_train"
TEST_PATH="/kaggle/input/asl-dataset-mine/archive/ASL_Alphabet_Dataset/asl_alphabet_test"

In [ ]:
# Reload raw data (0-255 integers)
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(96, 96),
    batch_size=32,
    label_mode='int' # IMPORTANT: Returns integers 0, 1, 2...
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(96, 96),
    batch_size=32,
    label_mode='int'
)

# Only performance optimizations, NO NORMALIZATION MAPPING
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

print("✅ Data reloaded. Pixel values should be 0-255.")

In [ ]:
model = Sequential([
    Input(shape=(96, 96, 3)),
    
    # NORMALIZATION HAPPENS HERE NOW
    # This turns 0-255 inputs into 0-1
    tf.keras.layers.Rescaling(1./255),
    
    # Simple CNN Architecture
    Conv2D(32, (3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(64, activation='relu'),
    
    # Output Layer
    Dense(num_classes, activation='softmax')
])

# Use standard learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
# Take 1 batch
batch_X, batch_y = next(iter(train_ds))

print("Training on single batch...")
# Train 20 times on the SAME 32 images
history = model.fit(batch_X, batch_y, epochs=20, verbose=1)

In [1]:
TRAIN_PATH="/kaggle/input/asl-dataset-mine/archive/ASL_Alphabet_Dataset/asl_alphabet_train"
TEST_PATH="/kaggle/input/asl-dataset-mine/archive/ASL_Alphabet_Dataset/asl_alphabet_test"

In [3]:
TRAIN_PATH="/kaggle/input/asl-updated-data/archive(1)/archive/ASL_Alphabet_Dataset/asl_alphabet_train"
TEST_PATH="/kaggle/input/asl-updated-data/archive(1)/archive/ASL_Alphabet_Dataset/asl_alphabet_test"

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, 
    BatchNormalization, GlobalAveragePooling2D, Input
)
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
)
from tensorflow.keras.regularizers import l2
import numpy as np

# ============================================================================
# MEMORY OPTIMIZATION SETTINGS
# ============================================================================

# Clear any previous models from memory
tf.keras.backend.clear_session()

# Enable memory growth for GPU (prevents allocation of all GPU memory at once)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU memory growth enabled for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(f"⚠️  {e}")

# Set TensorFlow to use less memory
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

print("\n" + "="*60)
print("MEMORY-EFFICIENT CONFIGURATION")
print("="*60)

# ============================================================================
# ANTI-OVERFITTING CONFIGURATION
# ============================================================================

# TRAIN_PATH = "/kaggle/input/american-sign-language/ASL_Dataset/Train"
# TEST_PATH = "/kaggle/input/american-sign-language/ASL_Dataset/Test"

batch_size = 32  # REDUCED from 32 - uses half the memory!
img_size = (96, 96)  # REDUCED from 128 - 30% less memory per image!

# ============================================================================
# MODERATE DATA AUGMENTATION (was too aggressive!)
# ============================================================================

data_augmentation = tf.keras.Sequential([
    # Moderate geometric transformations
    tf.keras.layers.RandomRotation(0.15),  # Reduced from 0.2
    tf.keras.layers.RandomTranslation(0.1, 0.1),  # Reduced from 0.15
    tf.keras.layers.RandomZoom(0.15),  # Reduced from 0.2
    tf.keras.layers.RandomFlip("horizontal"),
    
    # Moderate lighting variations
    tf.keras.layers.RandomBrightness(0.2, value_range=(0, 1)),  # SPECIFY RANGE!
    tf.keras.layers.RandomContrast(0.2),  # Reduced from 0.3
])

normalization = tf.keras.layers.Rescaling(1./255)

# ============================================================================
# LOAD DATA - MEMORY EFFICIENT VERSION
# ============================================================================

# CRITICAL: Don't load all data at once!
# Use generators to load batches on-the-fly

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,
    label_mode='int'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    label_mode='int'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH,
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")
print(f"Classes: {class_names}")

# Apply augmentation - NO CACHING (saves memory!)
def prepare_dataset(ds, augment=False):
    # Normalize first
    ds = ds.map(lambda x, y: (normalization(x), y), 
                num_parallel_calls=tf.data.AUTOTUNE)
    
    # Augment if training
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=tf.data.AUTOTUNE)
    
    # CRITICAL: Only prefetch, NO cache() to save memory
    return ds.prefetch(buffer_size=tf.data.AUTOTUNE)

train_ds = prepare_dataset(train_ds, augment=True)
val_ds = prepare_dataset(val_ds, augment=False)
# test_ds = prepare_dataset(test_ds, augment=False)

print("✅ Data loaded WITHOUT caching (memory efficient!)")

# ============================================================================
# VERIFY DATA IS PROPERLY NORMALIZED
# ============================================================================

print("\n" + "="*60)
print("VERIFYING DATA PREPROCESSING")
print("="*60)

for images, labels in train_ds.take(1):
    print(f"Batch shape: {images.shape}")
    print(f"Image dtype: {images.dtype}")
    print(f"Min value: {tf.reduce_min(images).numpy():.3f}")
    print(f"Max value: {tf.reduce_max(images).numpy():.3f}")
    print(f"Mean value: {tf.reduce_mean(images).numpy():.3f}")
    
    if tf.reduce_min(images) >= 0.0 and tf.reduce_max(images) <= 1.1:
        print("✅ Images properly normalized to [0, 1] range!")
    else:
        print("❌ ERROR: Images NOT normalized! Check preprocessing!")
        print("   Images should be in range [0, 1]")

print("="*60 + "\n")

# ============================================================================
# SOLUTION 1: TRANSFER LEARNING (BEST FOR OVERFITTING)
# ============================================================================

def create_transfer_learning_model(num_classes, trainable_layers=10):
    """
    MEMORY EFFICIENT: Smaller input + fewer trainable layers
    """
    
    # Load pre-trained model
    base_model = MobileNetV2(
        input_shape=(96, 96, 3),  # Smaller input size
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze MORE layers (less memory during training)
    base_model.trainable = True
    for layer in base_model.layers[:-trainable_layers]:  # Only train last 10 layers
        layer.trainable = False
    
    print(f"✅ MobileNetV2: Only {trainable_layers} layers trainable (memory efficient)")
    
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        
        # SMALLER dense layers
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),  # Reduced from 256
        Dropout(0.5),
        
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),  # Reduced from 128
        Dropout(0.4),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# ============================================================================
# SOLUTION 2: LIGHTWEIGHT CNN WITH HEAVY REGULARIZATION
# ============================================================================

def create_regularized_cnn(num_classes):
    """
    MEMORY EFFICIENT: Smaller, simpler CNN
    """
    
    model = Sequential([
        # Block 1
        Conv2D(32, (3,3), activation='relu', padding='same', 
               kernel_regularizer=l2(0.001), input_shape=(96,96,3)),  # 96x96 input
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.3),
        
        # Block 2 - SMALLER filters
        Conv2D(64, (3,3), activation='relu', padding='same', 
               kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.3),
        
        # Block 3 - Removed 4th block to save memory
        Conv2D(128, (3,3), activation='relu', padding='same', 
               kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.4),
        
        # Smaller classification head
        GlobalAveragePooling2D(),
        
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),  # Reduced from 256
        Dropout(0.5),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# ============================================================================
# SOLUTION 3: TINY MODEL (Harder to Overfit)
# ============================================================================

def create_tiny_model(num_classes):
    """
    ULTRA LIGHTWEIGHT - use if still crashing
    """
    
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(96,96,3)),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        GlobalAveragePooling2D(),
        
        Dense(64, activation='relu'),  # Very small
        Dropout(0.5),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# ============================================================================
# CREATE MODEL (Choose based on memory)
# ============================================================================

# Start with regularized CNN (better than tiny for learning)
model = create_regularized_cnn(num_classes)

# If this works well, try transfer learning for even better results
# model = create_transfer_learning_model(num_classes, trainable_layers=10)

# Only use tiny if you're still having memory issues
# model = create_tiny_model(num_classes)

print(f"\n✅ Model created with {model.count_params():,} parameters")

# ============================================================================
# COMPILE WITH LOWER LEARNING RATE (Prevents Overfitting)
# ============================================================================

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # INCREASED from 0.0001!
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ============================================================================
# ANTI-OVERFITTING CALLBACKS
# ============================================================================

# Stop early when validation stops improving
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Watch validation, not training
    patience=10,  # Stop if no improvement for 10 epochs
    restore_best_weights=True,
    verbose=1,
    mode='max'
)

# Save best model based on VALIDATION accuracy
model_checkpoint = ModelCheckpoint(
    'best_generalized_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Reduce learning rate when stuck
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

# ============================================================================
# TRAINING WITH VALIDATION MONITORING
# ============================================================================

print("\n" + "="*60)
print("TRAINING WITH ANTI-OVERFITTING STRATEGIES")
print("="*60)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    # Removed steps_per_epoch limit - let it train on full data
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    verbose=1
)

# ============================================================================
# DETECT OVERFITTING IN RESULTS
# ============================================================================

print("\n" + "="*60)
print("OVERFITTING CHECK")
print("="*60)

final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
gap = final_train_acc - final_val_acc

print(f"Final Training Accuracy:   {final_train_acc*100:.2f}%")
print(f"Final Validation Accuracy: {final_val_acc*100:.2f}%")
print(f"Gap (Train - Val):         {gap*100:.2f}%")

if gap > 0.15:  # More than 15% gap
    print("\n⚠️  HIGH OVERFITTING DETECTED!")
    print("Solutions:")
    print("  1. Add MORE data augmentation")
    print("  2. Use SMALLER model (fewer parameters)")
    print("  3. Increase Dropout rates")
    print("  4. Get more training data")
elif gap > 0.05:
    print("\n⚠️  Mild overfitting - manageable")
else:
    print("\n✅ Good generalization!")

# ============================================================================
# TEST ON COMPLETELY UNSEEN DATA (MOST IMPORTANT!)
# ============================================================================

print("\n" + "="*60)
print("REAL-WORLD TEST (Different people/conditions)")
print("="*60)

test_loss, test_accuracy = model.evaluate(test_ds)

print(f"\nTraining Accuracy:   {final_train_acc*100:.2f}%")
print(f"Validation Accuracy: {final_val_acc*100:.2f}%")
print(f"TEST Accuracy:       {test_accuracy*100:.2f}%  ← REAL PERFORMANCE")
print(f"{'='*60}")

# THIS IS THE NUMBER THAT MATTERS!
if test_accuracy < 0.7 and final_val_acc > 0.95:
    print("\n🚨 SEVERE DATA LEAKAGE DETECTED!")
    print("Val = {:.0f}% but Test = {:.0f}%".format(final_val_acc*100, test_accuracy*100))
    print("\nYour validation set is TOO SIMILAR to training!")
    print("\nSOLUTIONS:")
    print("  1. Get test images from DIFFERENT people")
    print("  2. Use DIFFERENT backgrounds")
    print("  3. Use DIFFERENT lighting conditions")
    print("  4. Use DIFFERENT cameras/phones")
    print("  5. Add natural variations (jewelry, sleeves, skin tones)")
elif test_accuracy < 0.85:
    print("\n⚠️  Model struggles with real-world variety")
    print("Need MORE diverse training data!")
else:
    print("\n✅ Good generalization to new data!")

# ============================================================================
# ANALYZE WHAT THE MODEL IS ACTUALLY LEARNING
# ============================================================================

print("\n" + "="*60)
print("CHECKING: Is the model learning HAND SHAPES or BACKGROUND?")
print("="*60)

# Sample some test predictions to see what's failing
sample_images = []
sample_labels = []
sample_preds = []

for batch_x, batch_y in test_ds.take(3):
    preds = model.predict(batch_x, verbose=0)
    sample_images.extend(batch_x.numpy()[:5])
    sample_labels.extend(batch_y.numpy()[:5])
    sample_preds.extend(np.argmax(preds[:5], axis=1))

print("\nSample Predictions:")
for i in range(min(15, len(sample_labels))):
    true_label = class_names[sample_labels[i]]
    pred_label = class_names[sample_preds[i]]
    status = "✅" if true_label == pred_label else "❌"
    print(f"  {status} True: {true_label:2s} | Predicted: {pred_label:2s}")

# ============================================================================
# VISUALIZATION: Spot Overfitting
# ============================================================================

import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot - should see train/val stay close together
ax[0].plot(history.history['accuracy'], label='Train Acc', linewidth=2)
ax[0].plot(history.history['val_accuracy'], label='Val Acc', linewidth=2)
ax[0].set_title("Training vs Validation Accuracy", fontsize=14, fontweight='bold')
ax[0].set_xlabel("Epoch", fontsize=12)
ax[0].set_ylabel("Accuracy", fontsize=12)
ax[0].legend(fontsize=11)
ax[0].grid(True, alpha=0.3)

# Add annotation if overfitting
if gap > 0.15:
    ax[0].text(0.5, 0.5, 'OVERFITTING!', 
               transform=ax[0].transAxes, 
               fontsize=20, color='red', alpha=0.3,
               ha='center', va='center')

# Loss plot - validation should decrease smoothly
ax[1].plot(history.history['loss'], label='Train Loss', linewidth=2)
ax[1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
ax[1].set_title("Training vs Validation Loss", fontsize=14, fontweight='bold')
ax[1].set_xlabel("Epoch", fontsize=12)
ax[1].set_ylabel("Loss", fontsize=12)
ax[1].legend(fontsize=11)
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("overfitting_check.png", dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# DETAILED METRICS
# ============================================================================

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

y_pred = []
y_true = []

for batch_x, batch_y in test_ds:
    preds = model.predict(batch_x, verbose=0)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(batch_y.numpy())

y_pred = np.array(y_pred)
y_true = np.array(y_true)

print("\nPer-Class Performance (find weak classes):")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(16, 14))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Generalized Model')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('generalized_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# SAVE BEST MODEL
# ============================================================================

model.save("asl_generalized_model_1.keras")
print("\n✅ Generalized model saved!")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("asl_generalized.tflite", "wb") as f:
    f.write(tflite_model)
    
print("✅ TFLite model saved!")

# ============================================================================
# ADDITIONAL TIPS FOR 100% VAL BUT BAD REAL-WORLD PERFORMANCE
# ============================================================================

print("\n" + "="*60)
print("WHY 100% VAL BUT FAILS ON NEW IMAGES?")
print("="*60)
print("\n🔍 Most Common Causes:")
print("  1. SAME PERSON in train/val/test")
print("     → Model memorizes YOUR hand, not hand shapes")
print("\n  2. SAME BACKGROUND (white wall, desk, etc)")
print("     → Model uses background to classify, not hand!")
print("\n  3. SAME LIGHTING/CAMERA")
print("     → Model memorizes photo conditions")
print("\n  4. INSUFFICIENT VARIETY")
print("     → Need multiple people, skin tones, angles")
print("\n✅ SOLUTIONS:")
print("  1. Collect images from 5+ different people")
print("  2. Use varied backgrounds (blur them if possible)")
print("  3. Different lighting (indoor, outdoor, bright, dim)")
print("  4. Different hand positions/rotations")
print("  5. Include hand accessories (rings, watches, sleeves)")
print("  6. Use data from multiple cameras/phones")
print("\n💡 QUICK TEST:")
print("  Take 10 photos of YOUR OWN hand right now")
print("  → If model gets <50% accuracy, it's memorizing!")
print("="*60)

# ============================================================================
# EMERGENCY FIX: Background Randomization
# ============================================================================

print("\n" + "="*60)
print("EMERGENCY AUGMENTATION (if can't get new data)")
print("="*60)

# Add this to your data_augmentation
extreme_augmentation = tf.keras.Sequential([
    # Original augmentations
    tf.keras.layers.RandomRotation(0.3),  # Even more rotation
    tf.keras.layers.RandomTranslation(0.2, 0.2),
    tf.keras.layers.RandomZoom(0.3),
    tf.keras.layers.RandomFlip("horizontal"),
    
    # EXTREME lighting changes
    tf.keras.layers.RandomBrightness(0.4),  # Bigger range
    tf.keras.layers.RandomContrast(0.5),
    
    # Color distortion to ignore background colors
    # This forces model to focus on hand shape, not colors
    tf.keras.layers.Lambda(lambda x: tf.image.random_hue(x, 0.2)),
    tf.keras.layers.Lambda(lambda x: tf.image.random_saturation(x, 0.5, 1.5)),
])

print("Added EXTREME augmentation to break background memorization")

✅ GPU memory growth enabled for 2 GPU(s)

MEMORY-EFFICIENT CONFIGURATION
Found 223325 files belonging to 29 classes.
Using 178660 files for training.
Found 223325 files belonging to 29 classes.
Using 44665 files for validation.
Found 28 files belonging to 28 classes.
Number of classes: 29
Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
✅ Data loaded WITHOUT caching (memory efficient!)

VERIFYING DATA PREPROCESSING
Batch shape: (32, 96, 96, 3)
Image dtype: <dtype: 'float32'>
Min value: 0.000
Max value: 1.087
Mean value: 0.528
✅ Images properly normalized to [0, 1] range!



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



✅ Model created with 114,397 parameters


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 96, 96, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 96, 96, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...


TRAINING WITH ANTI-OVERFITTING STRATEGIES
Epoch 1/50


I0000 00:00:1765814589.004721     772 service.cc:148] XLA service 0x7cfba8018e00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765814589.004760     772 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1765814589.004765     772 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1765814589.664289     772 cuda_dnn.cc:529] Loaded cuDNN version 90300


   4/5584 ━━━━━━━━━━━━━━━━━━━━ 1:59 21ms/step - accuracy: 0.0195 - loss: 4.2645       

I0000 00:00:1765814596.863821     772 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2711/5584 ━━━━━━━━━━━━━━━━━━━━ 2:34 54ms/step - accuracy: 0.0756 - loss: 3.4082

In [ ]:
TRAIN_PATH="/kaggle/input/asl-dataset-mine/archive/ASL_Alphabet_Dataset/asl_alphabet_train"
TEST_PATH="/kaggle/input/asl-dataset-mine/archive/ASL_Alphabet_Dataset/asl_alphabet_test"

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, 
    BatchNormalization, GlobalAveragePooling2D, Input
)
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
)
from tensorflow.keras.regularizers import l2
import numpy as np

# ============================================================================
# MEMORY OPTIMIZATION SETTINGS
# ============================================================================

tf.keras.backend.clear_session()

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU memory growth enabled for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(f"⚠️  {e}")

import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'  # Better GPU threading
os.environ['TF_GPU_THREAD_COUNT'] = '2'

print("\n" + "="*60)
print("MEMORY-EFFICIENT CONFIGURATION + PIPELINE FIX")
print("="*60)

# ============================================================================
# CRITICAL: REDUCED BATCH SIZE & OPTIMIZED SETTINGS
# ============================================================================

batch_size = 8  # REDUCED from 32 - less I/O pressure
img_size = (96, 96)

# ============================================================================
# SIMPLIFIED AUGMENTATION (Less CPU overhead)
# ============================================================================

# CRITICAL: Make augmentation MUCH simpler to reduce CPU load
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.15),
    tf.keras.layers.RandomTranslation(0.1, 0.1),
    tf.keras.layers.RandomZoom(0.15),
    tf.keras.layers.RandomFlip("horizontal"),
])

normalization = tf.keras.layers.Rescaling(1./255)

# ============================================================================
# OPTIMIZED DATA LOADING - NO MORE STALLS!
# ============================================================================

print("Loading datasets with optimized pipeline...")

# CRITICAL: Use more parallel calls and better caching strategy
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,
    label_mode='int'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    label_mode='int'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH,
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")
print(f"Classes: {class_names}")

# ============================================================================
# CRITICAL FIX: OPTIMIZED PIPELINE TO PREVENT STALLING
# ============================================================================

def prepare_dataset(ds, augment=False, cache_data=False):
    """
    OPTIMIZED: Better ordering and parallelization
    """
    # Step 1: Cache BEFORE augmentation (if dataset is small enough)
    if cache_data:
        ds = ds.cache()  # Cache the raw images
        print("✅ Dataset cached in memory")
    
    # Step 2: Normalize (fast operation)
    ds = ds.map(lambda x, y: (normalization(x), y), 
                num_parallel_calls=AUTOTUNE)
    
    # Step 3: Augment if training (CPU-intensive, so parallelize heavily)
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=AUTOTUNE,
                    deterministic=False)  # Allow parallel non-deterministic processing
    
    # Step 4: CRITICAL - Large prefetch buffer
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    
    return ds

# Determine if we can cache (check dataset size)
# If your dataset is < 5GB, caching helps A LOT
dataset_size_estimate = len(list(train_ds)) * batch_size * 96 * 96 * 3 * 4 / (1024**3)
print(f"Estimated train dataset size: {dataset_size_estimate:.2f} GB")

can_cache = dataset_size_estimate < 3  # Only cache if < 3GB

if can_cache:
    print("✅ Dataset small enough - will cache for speed")
else:
    print("⚠️ Dataset too large to cache - using streaming mode")

train_ds = prepare_dataset(train_ds, augment=True, cache_data=can_cache)
val_ds = prepare_dataset(val_ds, augment=False, cache_data=can_cache)
test_ds = prepare_dataset(test_ds, augment=False, cache_data=False)

print("✅ Optimized data pipeline configured!")

# ============================================================================
# VERIFY DATA PIPELINE IS WORKING
# ============================================================================

print("\n" + "="*60)
print("TESTING DATA PIPELINE (checking for stalls)")
print("="*60)

import time

# Test loading speed
start = time.time()
batch_count = 0
for images, labels in train_ds.take(50):  # Test 50 batches
    batch_count += 1
    if batch_count % 10 == 0:
        print(f"  Loaded {batch_count} batches... ({time.time() - start:.2f}s)")

elapsed = time.time() - start
batches_per_sec = batch_count / elapsed

print(f"\n✅ Pipeline test complete:")
print(f"   {batch_count} batches in {elapsed:.2f}s")
print(f"   Speed: {batches_per_sec:.2f} batches/sec")

if batches_per_sec < 1.0:
    print("\n⚠️ WARNING: Pipeline is slow! This will cause stalls.")
    print("   Solutions:")
    print("   1. Reduce batch_size further (try 8)")
    print("   2. Simplify augmentation even more")
    print("   3. Use smaller image size (64x64)")
else:
    print("\n✅ Pipeline speed looks good!")

print("="*60 + "\n")

# ============================================================================
# MODEL CREATION
# ============================================================================

def create_regularized_cnn(num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', 
               kernel_regularizer=l2(0.001), input_shape=(96,96,3)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.3),
        
        Conv2D(64, (3,3), activation='relu', padding='same', 
               kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.3),
        
        Conv2D(128, (3,3), activation='relu', padding='same', 
               kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.4),
        
        GlobalAveragePooling2D(),
        
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.5),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

model = create_regularized_cnn(num_classes)
print(f"\n✅ Model created with {model.count_params():,} parameters")

# ============================================================================
# COMPILE
# ============================================================================

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ============================================================================
# CALLBACKS WITH PROGRESS MONITORING
# ============================================================================

class StallDetector(tf.keras.callbacks.Callback):
    """Detect if training is stalling"""
    def __init__(self):
        super().__init__()
        self.last_batch_time = time.time()
        self.batch_times = []
    
    def on_batch_begin(self, batch, logs=None):
        self.last_batch_time = time.time()
    
    def on_batch_end(self, batch, logs=None):
        batch_time = time.time() - self.last_batch_time
        self.batch_times.append(batch_time)
        
        # Alert if batch takes too long
        if batch_time > 10.0:  # More than 10 seconds per batch
            print(f"\n⚠️ SLOW BATCH DETECTED: {batch_time:.1f}s for batch {batch}")
    
    def on_epoch_end(self, epoch, logs=None):
        if len(self.batch_times) > 0:
            avg_time = np.mean(self.batch_times)
            print(f"\n📊 Avg batch time: {avg_time:.3f}s")
            self.batch_times = []  # Reset for next epoch

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=15,
    restore_best_weights=True,
    verbose=1,
    mode='max'
)

model_checkpoint = ModelCheckpoint(
    'best_generalized_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

stall_detector = StallDetector()

# ============================================================================
# TRAINING WITH STALL DETECTION
# ============================================================================

print("\n" + "="*60)
print("TRAINING WITH STALL DETECTION")
print("="*60)

try:
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=50,
        callbacks=[early_stopping, model_checkpoint, reduce_lr, stall_detector],
        verbose=1,
        workers=4,  # Parallel data loading
        use_multiprocessing=False  # Set to False on Kaggle to avoid issues
    )
except KeyboardInterrupt:
    print("\n⚠️ Training interrupted by user")
except Exception as e:
    print(f"\n❌ Training failed: {e}")
    raise

# ============================================================================
# RESULTS
# ============================================================================

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)

final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
gap = final_train_acc - final_val_acc

print(f"Final Training Accuracy:   {final_train_acc*100:.2f}%")
print(f"Final Validation Accuracy: {final_val_acc*100:.2f}%")
print(f"Gap (Train - Val):         {gap*100:.2f}%")

# Test evaluation
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy:             {test_accuracy*100:.2f}%")

# ============================================================================
# SAVE MODEL
# ============================================================================

model.save("asl_generalized_model_fixed.keras")
print("\n✅ Model saved!")

# ============================================================================
# EMERGENCY TROUBLESHOOTING
# ============================================================================

print("\n" + "="*60)
print("IF STILL EXPERIENCING STALLS:")
print("="*60)
print("\n1. REDUCE BATCH SIZE EVEN MORE:")
print("   batch_size = 8  # or even 4")
print("\n2. SIMPLIFY AUGMENTATION:")
print("   Remove RandomBrightness and RandomContrast")
print("\n3. REDUCE IMAGE SIZE:")
print("   img_size = (64, 64)")
print("\n4. CHECK KAGGLE RESOURCES:")
print("   !nvidia-smi  # Check GPU usage")
print("   !free -h     # Check RAM usage")
print("\n5. RESTART KERNEL:")
print("   Sometimes Kaggle sessions get corrupted")
print("="*60)

In [ ]:
TRAIN_PATH="/kaggle/input/asl-dataset-mine/archive/ASL_Alphabet_Dataset/asl_alphabet_train"
TEST_PATH="/kaggle/input/asl-dataset-mine/archive/ASL_Alphabet_Dataset/asl_alphabet_test"

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np

# ============================================================================
# NUCLEAR MEMORY FIX - THIS WILL WORK
# ============================================================================

print("🔥 ULTRA-LOW MEMORY MODE ACTIVATED 🔥\n")

# Clear everything
tf.keras.backend.clear_session()
import gc
gc.collect()

# CRITICAL: Prevent TensorFlow from being greedy
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'  # Better memory management
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# GPU settings - CRUCIAL for preventing OOM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Limit GPU memory to 4GB (adjust if needed)
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=4096)]
        )
        print(f"✅ GPU memory limited to 4GB")
    except:
        # Fallback: just enable memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU memory growth enabled")

# ============================================================================
# MINIMAL SETTINGS - NO MORE BS
# ============================================================================

BATCH_SIZE = 8  # TINY batch size
IMG_SIZE = (64, 64)  # SMALLER images (64x64 instead of 96x96)
PREFETCH = 2  # Minimal prefetch

print(f"Batch size: {BATCH_SIZE}")
print(f"Image size: {IMG_SIZE}")

# ============================================================================
# ULTRA-SIMPLE AUGMENTATION (Almost nothing)
# ============================================================================

# Just normalization and basic flip - NO heavy augmentation
def preprocess_image(image, label, augment=False):
    # Normalize
    image = tf.cast(image, tf.float32) / 255.0
    
    # Only basic augmentation if training
    if augment:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_brightness(image, 0.1)
    
    return image, label

# ============================================================================
# LOAD DATA - STREAMING MODE (NO CACHING!)
# ============================================================================

print("\nLoading data in streaming mode (no cache)...")

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    label_mode='int'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
    label_mode='int'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH,
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"✅ {num_classes} classes loaded: {class_names[:5]}...")

# ============================================================================
# APPLY PREPROCESSING - NO CACHE, MINIMAL PREFETCH
# ============================================================================

# Apply preprocessing
train_ds = train_ds.map(
    lambda x, y: preprocess_image(x, y, augment=True),
    num_parallel_calls=1  # SINGLE thread to save memory
).prefetch(PREFETCH)

val_ds = val_ds.map(
    lambda x, y: preprocess_image(x, y, augment=False),
    num_parallel_calls=1
).prefetch(PREFETCH)

test_ds = test_ds.map(
    lambda x, y: preprocess_image(x, y, augment=False),
    num_parallel_calls=1
).prefetch(PREFETCH)

print("✅ Data pipeline ready (minimal memory)\n")

# ============================================================================
# TINY MODEL - Can't OOM if model is small
# ============================================================================

def create_tiny_model(num_classes):
    """
    Smallest model that still learns
    ~50K parameters vs millions
    """
    model = Sequential([
        # Block 1
        Conv2D(16, (3,3), activation='relu', padding='same', input_shape=(64,64,3)),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        # Block 2
        Conv2D(32, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        # Block 3
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        # Classifier
        GlobalAveragePooling2D(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

model = create_tiny_model(num_classes)
print(f"Model parameters: {model.count_params():,}")
print(f"Estimated model size: {model.count_params() * 4 / 1024 / 1024:.1f} MB\n")

# ============================================================================
# COMPILE
# ============================================================================

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ============================================================================
# CALLBACKS - MINIMAL
# ============================================================================

callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_model_tiny.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]

# ============================================================================
# TRAIN WITH MANUAL GARBAGE COLLECTION
# ============================================================================

print("\n" + "="*60)
print("STARTING TRAINING - MINIMAL MEMORY MODE")
print("="*60 + "\n")

# Train in smaller chunks if needed
try:
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=50,
        callbacks=callbacks,
        verbose=1
    )
    
    print("\n✅ TRAINING COMPLETED!")
    
except tf.errors.ResourceExhaustedError as e:
    print(f"\n❌ STILL OUT OF MEMORY!")
    print(f"Error: {e}")
    print("\nTRY THESE:")
    print("1. Restart kernel completely")
    print("2. Change BATCH_SIZE to 4")
    print("3. Change IMG_SIZE to (48, 48)")
    print("4. Use CPU instead: os.environ['CUDA_VISIBLE_DEVICES'] = '-1'")
    raise

# ============================================================================
# EVALUATE
# ============================================================================

print("\n" + "="*60)
print("EVALUATION")
print("="*60)

final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]

print(f"\nTraining Accuracy:   {final_train_acc*100:.2f}%")
print(f"Validation Accuracy: {final_val_acc*100:.2f}%")

test_loss, test_accuracy = model.evaluate(test_ds, verbose=0)
print(f"Test Accuracy:       {test_accuracy*100:.2f}%")

# ============================================================================
# SAVE
# ============================================================================

model.save("asl_tiny_model.keras")
print("\n✅ Model saved!")

# Convert to TFLite (MUCH smaller)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("asl_tiny.tflite", "wb") as f:
    f.write(tflite_model)

print(f"✅ TFLite model saved! ({len(tflite_model) / 1024:.1f} KB)")

# ============================================================================
# PLOT RESULTS
# ============================================================================

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "="*60)
print("IF THIS STILL FAILS:")
print("="*60)
print("\n1. RESTART KERNEL (Runtime -> Restart Runtime)")
print("2. Run ONLY this script (no other code)")
print("3. If still fails, add this at the TOP:")
print("   os.environ['CUDA_VISIBLE_DEVICES'] = '-1'")
print("   (This forces CPU training)")
print("="*60)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np

# ============================================================================
# MEMORY OPTIMIZATION
# ============================================================================

print("🚀 IMPROVED ASL MODEL - TARGET: 95%+ ACCURACY\n")

tf.keras.backend.clear_session()
import gc
gc.collect()

import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=4096)]
        )
        print(f"✅ GPU memory limited to 4GB")
    except:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU memory growth enabled")

# ============================================================================
# IMPROVED SETTINGS
# ============================================================================

BATCH_SIZE = 16  # Slightly larger for stability
IMG_SIZE = (96, 96)  # Larger image = more detail (changed from 64x64)
PREFETCH = 3

print(f"Batch size: {BATCH_SIZE}")
print(f"Image size: {IMG_SIZE}")

# ============================================================================
# BETTER AUGMENTATION - Key to 95%+
# ============================================================================

# ============================================================================
# DIRTY DATA AUGMENTATION (Simulates Webcams)
# ============================================================================

def tough_augmentation(x, training=True):
    if not training:
        return x
    
    # 1. Geometry (Standard)
    x = tf.image.random_rotation(x, 0.15)  # Less rotation, hands don't spin 360
    x = tf.image.random_crop(x, [BATCH_SIZE, IMG_SIZE[0], IMG_SIZE[1], 3]) # Simulate imperfect tracking
    
    # 2. Lighting/Color (CRITICAL for video)
    x = tf.image.random_brightness(x, 0.2)
    x = tf.image.random_contrast(x, 0.8, 1.2)
    x = tf.image.random_saturation(x, 0.7, 1.3) # Webcams wash out colors
    
    # 3. Simulate low quality/Blur (The Secret Sauce)
    # random_jpeg_quality introduces artifacts common in video streams
    x = tf.image.random_jpeg_quality(x, 70, 100) 
    
    return x

# Update prepare function
def prepare(ds, shuffle=False, augment=False):
    if shuffle:
        ds = ds.shuffle(1000)
    
    # Normalize
    ds = ds.map(lambda x, y: (normalization(x), y))
    
    # Use custom augmentation
    if augment:
        ds = ds.map(lambda x, y: (tough_augmentation(x, training=True), y))
    
    return ds.prefetch(2)
# ============================================================================
# LOAD DATA
# ============================================================================

print("\nLoading data...")

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    label_mode='int'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
    label_mode='int'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH,
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"✅ {num_classes} classes loaded")

# ============================================================================
# APPLY PREPROCESSING
# ============================================================================

train_ds = train_ds.map(
    lambda x, y: preprocess_image(x, y, augment=True),
    num_parallel_calls=tf.data.AUTOTUNE
).prefetch(PREFETCH)

val_ds = val_ds.map(
    lambda x, y: preprocess_image(x, y, augment=False),
    num_parallel_calls=tf.data.AUTOTUNE
).prefetch(PREFETCH)

test_ds = test_ds.map(
    lambda x, y: preprocess_image(x, y, augment=False),
    num_parallel_calls=tf.data.AUTOTUNE
).prefetch(PREFETCH)

print("✅ Data pipeline ready\n")

# ============================================================================
# IMPROVED MODEL - Deeper but still memory-efficient
# ============================================================================

def create_improved_model(num_classes):
    """
    Better model: deeper + BatchNorm for 95%+ accuracy
    Still under 500K parameters to avoid memory issues
    """
    model = Sequential([
        # Block 1
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(96,96,3)),
        BatchNormalization(),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        # Block 2
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        # Block 3
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.3),
        
        # Block 4 - Extra depth for complex patterns
        Conv2D(256, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.4),
        
        # Classifier
        GlobalAveragePooling2D(),
        
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(128, activation='relu'),
        Dropout(0.4),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

model = create_improved_model(num_classes)
print(f"Model parameters: {model.count_params():,}")
print(f"Estimated size: {model.count_params() * 4 / 1024 / 1024:.1f} MB\n")

# ============================================================================
# COMPILE WITH BETTER OPTIMIZER
# ============================================================================

# Use AdamW (Adam with weight decay) for better generalization
model.compile(
    optimizer=tf.keras.optimizers.AdamW(
        learning_rate=0.001,
        weight_decay=0.0001  # Helps prevent overfitting
    ),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ============================================================================
# CALLBACKS - TUNED FOR PERFORMANCE
# ============================================================================

callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,  # More patience for better convergence
        restore_best_weights=True,
        verbose=1,
        min_delta=0.001  # Only stop if no 0.1% improvement
    ),
    ModelCheckpoint(
        'best_asl_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

# ============================================================================
# TRAIN
# ============================================================================

print("\n" + "="*60)
print("TRAINING - TARGET: 95%+ ACCURACY")
print("="*60 + "\n")

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,  # More epochs with early stopping
    callbacks=callbacks,
    verbose=1
)

print("\n✅ TRAINING COMPLETED!")

# ============================================================================
# EVALUATE
# ============================================================================

print("\n" + "="*60)
print("RESULTS")
print("="*60)

final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
gap = final_train_acc - final_val_acc

print(f"\nTraining Accuracy:   {final_train_acc*100:.2f}%")
print(f"Validation Accuracy: {final_val_acc*100:.2f}%")
print(f"Gap (Train - Val):   {gap*100:.2f}%")

test_loss, test_accuracy = model.evaluate(test_ds, verbose=0)
print(f"Test Accuracy:       {test_accuracy*100:.2f}%")

if test_accuracy >= 0.95:
    print("\n🎉 EXCELLENT! 95%+ achieved!")
elif test_accuracy >= 0.92:
    print("\n✅ Very good! Close to target.")
else:
    print("\n📈 Good progress! See tips below for 95%+")

# ============================================================================
# ANALYZE MISTAKES
# ============================================================================

print("\n" + "="*60)
print("ANALYZING MISTAKES")
print("="*60)

from sklearn.metrics import classification_report, confusion_matrix

y_pred = []
y_true = []

for batch_x, batch_y in test_ds:
    preds = model.predict(batch_x, verbose=0)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(batch_y.numpy())

y_pred = np.array(y_pred)
y_true = np.array(y_true)

# Per-class accuracy
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

print("\n📊 Per-class Accuracy:")
class_accs = [(name, report[name]['precision']) for name in class_names]
class_accs.sort(key=lambda x: x[1])

print("\n🔴 WORST performing classes (focus on these):")
for name, acc in class_accs[:5]:
    print(f"  {name}: {acc*100:.1f}%")

print("\n🟢 BEST performing classes:")
for name, acc in class_accs[-5:]:
    print(f"  {name}: {acc*100:.1f}%")

# ============================================================================
# CONFUSION MATRIX
# ============================================================================

import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(20, 18))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Find Similar Signs That Get Confused', fontsize=16)
plt.ylabel('True Label', fontsize=14)
plt.xlabel('Predicted Label', fontsize=14)
plt.tight_layout()
plt.savefig('confusion_matrix_improved.png', dpi=200, bbox_inches='tight')
plt.show()

# ============================================================================
# TRAINING HISTORY
# ============================================================================

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

ax[0].plot(history.history['accuracy'], label='Train', linewidth=2)
ax[0].plot(history.history['val_accuracy'], label='Val', linewidth=2)
ax[0].set_title("Accuracy Over Time", fontsize=14, fontweight='bold')
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Accuracy")
ax[0].legend()
ax[0].grid(True, alpha=0.3)

ax[1].plot(history.history['loss'], label='Train', linewidth=2)
ax[1].plot(history.history['val_loss'], label='Val', linewidth=2)
ax[1].set_title("Loss Over Time", fontsize=14, fontweight='bold')
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Loss")
ax[1].legend()
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history_improved.png', dpi=200, bbox_inches='tight')
plt.show()

# ============================================================================
# SAVE MODEL
# ============================================================================

model.save("asl_improved_model.keras")
print("\n✅ Model saved!")

# TFLite conversion
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("asl_improved.tflite", "wb") as f:
    f.write(tflite_model)

print(f"✅ TFLite model saved! ({len(tflite_model) / 1024:.1f} KB)")

# ============================================================================
# TIPS TO REACH 95%+
# ============================================================================

print("\n" + "="*60)
print("HOW TO REACH 95%+ ACCURACY")
print("="*60)

print("\n🎯 If you're at 89-92%:")
print("  1. ✅ DONE: Better model architecture (this script)")
print("  2. ✅ DONE: Stronger augmentation")
print("  3. ✅ DONE: BatchNormalization for stability")

print("\n🎯 To reach 95%+, you need:")
print("\n  📸 MORE DATA:")
print("     - Collect 2x more images per class")
print("     - Get images from 5+ different people")
print("     - Use different backgrounds/lighting")

print("\n  🔍 FIX CONFUSING CLASSES:")
print("     - Look at confusion matrix above")
print("     - Similar signs (M/N, K/V) need MORE examples")
print("     - Add 2x data for the worst 5 classes")

print("\n  🏗️ TRY TRANSFER LEARNING:")
print("     - Use pre-trained MobileNetV2 (better features)")
print("     - Will get you to 95%+ with same data")

print("\n  ⚡ ADVANCED TECHNIQUES:")
print("     - Mixup augmentation (blend two images)")
print("     - Test-time augmentation (predict on 5 augmented versions)")
print("     - Ensemble (combine 3 models)")

print("\n💡 QUICK WIN:")
print("   Run this script 3 times, save each model")
print("   Average their predictions (ensemble)")
print("   +2-3% accuracy boost for free!")

print("="*60)

In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
TRAIN_PATH="/kaggle/input/asl-updated-data/archive(1)/archive/ASL_Alphabet_Dataset/asl_alphabet_train"
TEST_PATH="/kaggle/input/asl-updated-data/archive(1)/archive/ASL_Alphabet_Dataset/asl_alphabet_test"

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
import time

# ============================================================================
# NUCLEAR MODE - NO MORE BULLSHIT
# ============================================================================

print("💥 NUCLEAR MODE - FAST & STABLE 💥\n")

# RESTART EVERYTHING
tf.keras.backend.clear_session()
import gc
gc.collect()

# FORCE STABILITY
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Limit GPU to prevent OOM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=3072)]  # Only 3GB
        )
        print("✅ GPU limited to 3GB (stable)")
    except:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ Memory growth enabled")

# ============================================================================
# MINIMAL SETTINGS - GUARANTEED TO WORK
# ============================================================================

BATCH_SIZE = 8  # SMALL - won't stall
IMG_SIZE = (64, 64)  # SMALL - fast loading
EPOCHS = 50  # Enough for 8 hours

print(f"Batch: {BATCH_SIZE} | Image: {IMG_SIZE} | Epochs: {EPOCHS}\n")

# ============================================================================
# SIMPLE AUGMENTATION - NO FANCY SHIT
# ============================================================================

augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1),
], name="augmentation")

# ============================================================================
# LOAD DATA - STREAMING ONLY
# ============================================================================

print("Loading data (streaming mode)...")

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH,
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"✅ {num_classes} classes")

# ============================================================================
# NORMALIZE ONLY - KEEP IT SIMPLE
# ============================================================================

normalization = tf.keras.layers.Rescaling(1./255)

def prepare(ds, shuffle=False, augment=False):
    if shuffle:
        ds = ds.shuffle(1000)
    
    # Normalize
    ds = ds.map(lambda x, y: (normalization(x), y))
    
    # Augment AFTER normalize
    if augment:
        ds = ds.map(lambda x, y: (augmentation(x, training=True), y))
    
    # Minimal prefetch
    return ds.prefetch(2)

train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

print("✅ Pipeline ready\n")

# ============================================================================
# TEST PIPELINE SPEED (CRITICAL!)
# ============================================================================

print("Testing pipeline speed...")
start = time.time()
for i, (images, labels) in enumerate(train_ds.take(20)):
    if i % 5 == 0:
        print(f"  Batch {i+1}/20... {time.time()-start:.1f}s")
elapsed = time.time() - start
print(f"✅ 20 batches in {elapsed:.1f}s ({20/elapsed:.1f} batch/sec)")

if elapsed > 30:
    print("⚠️ WARNING: Pipeline is SLOW - training will stall!")
    print("   Reduce batch_size to 4 or restart kernel")
else:
    print("✅ Pipeline speed OK!\n")

# ============================================================================
# SOLID MODEL - NOT TOO BIG, NOT TOO SMALL
# ============================================================================

model = Sequential([
    # Block 1
    Conv2D(32, 3, activation='relu', padding='same', input_shape=(64,64,3)),
    BatchNormalization(),
    Conv2D(32, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.25),
    
    # Block 2
    Conv2D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.25),
    
    # Block 3
    Conv2D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.3),
    
    # Classifier
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

print(f"Model: {model.count_params():,} params")
print(f"Size: {model.count_params()*4/1024/1024:.1f} MB\n")

# ============================================================================
# COMPILE
# ============================================================================

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ============================================================================
# CALLBACKS WITH PROGRESS TRACKING
# ============================================================================

class ProgressTracker(tf.keras.callbacks.Callback):
    def __init__(self):
        self.start_time = time.time()
        self.epoch_start = None
    
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()
        elapsed = (time.time() - self.start_time) / 3600
        remaining = 8 - elapsed
        print(f"\n⏰ Epoch {epoch+1}/{EPOCHS} | Time used: {elapsed:.1f}h | Remaining: {remaining:.1f}h")
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_time = (time.time() - self.epoch_start) / 60
        print(f"✅ Epoch took {epoch_time:.1f} min | Val Acc: {logs['val_accuracy']*100:.2f}%")

callbacks = [
    ProgressTracker(),
    
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    
    ModelCheckpoint(
        'best_asl.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

# ============================================================================
# TRAIN WITH ERROR HANDLING
# ============================================================================

print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

try:
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=2  # Less verbose = faster
    )
    
    print("\n✅ TRAINING COMPLETED!")
    
except tf.errors.ResourceExhaustedError as e:
    print("\n❌ OUT OF MEMORY!")
    print("EMERGENCY FIX:")
    print("1. Runtime -> Restart Runtime")
    print("2. Change BATCH_SIZE = 4")
    print("3. Change IMG_SIZE = (48, 48)")
    raise

except KeyboardInterrupt:
    print("\n⚠️ Training stopped by user")
    print("Saving current model...")
    model.save("interrupted_model.keras")

except Exception as e:
    print(f"\n❌ ERROR: {e}")
    raise

# ============================================================================
# RESULTS
# ============================================================================

print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)

train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]

print(f"\nTrain Accuracy: {train_acc*100:.2f}%")
print(f"Val Accuracy:   {val_acc*100:.2f}%")

# Test
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"Test Accuracy:  {test_acc*100:.2f}%")

if test_acc >= 0.95:
    print("\n🎉 EXCELLENT! 95%+")
elif test_acc >= 0.90:
    print("\n✅ GOOD! 90%+")
elif test_acc >= 0.85:
    print("\n👍 Decent! 85%+")
else:
    print("\n📈 Needs improvement")

# ============================================================================
# ANALYZE WORST CLASSES
# ============================================================================

from sklearn.metrics import classification_report

y_pred, y_true = [], []
for x, y in test_ds:
    preds = model.predict(x, verbose=0)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(y.numpy())

report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

print("\n" + "="*60)
print("WORST 5 CLASSES (collect more data for these)")
print("="*60)

class_accs = [(name, report[name]['f1-score']) for name in class_names]
class_accs.sort(key=lambda x: x[1])

for name, score in class_accs[:5]:
    print(f"  {name}: {score*100:.1f}%")

# ============================================================================
# SAVE
# ============================================================================

model.save("asl_final_model.keras")
print("\n✅ Model saved: asl_final_model.keras")

# TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("asl_final.tflite", "wb") as f:
    f.write(tflite_model)

print(f"✅ TFLite saved: {len(tflite_model)/1024:.1f} KB")

# ============================================================================
# TRAINING PLOT
# ============================================================================

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('training_plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ DONE!")

# ============================================================================
# IF IT STALLS AGAIN
# ============================================================================

print("\n" + "="*60)
print("IF IT STALLS AT EPOCH 8 AGAIN:")
print("="*60)
print("\n1. RESTART KERNEL (Runtime -> Restart)")
print("2. Clear all outputs (Edit -> Clear all outputs)")
print("3. Run ONLY this cell")
print("4. If still fails:")
print("   BATCH_SIZE = 4")
print("   IMG_SIZE = (48, 48)")
print("\n5. NUCLEAR OPTION - Force CPU:")
print("   os.environ['CUDA_VISIBLE_DEVICES'] = '-1'")
print("   (Add at very top before imports)")
print("="*60)

In [1]:
TRAIN_PATH="/kaggle/input/asl-updated-data/archive(1)/archive/ASL_Alphabet_Dataset/asl_alphabet_train"
TEST_PATH="/kaggle/input/asl-updated-data/archive(1)/archive/ASL_Alphabet_Dataset/asl_alphabet_test"

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
import time

# ============================================================================
# NUCLEAR MODE - NO MORE BULLSHIT
# ============================================================================

print("💥 NUCLEAR MODE - FAST & STABLE 💥\n")

# RESTART EVERYTHING
tf.keras.backend.clear_session()
import gc
gc.collect()

# FORCE STABILITY
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Limit GPU to prevent OOM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=3072)]  # Only 3GB
        )
        print("✅ GPU limited to 3GB (stable)")
    except:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ Memory growth enabled")

# ============================================================================
# SETTINGS - TUNED FOR VIDEO SURVIVAL
# ============================================================================

BATCH_SIZE = 8   # Kept small for stability
IMG_SIZE = (96, 96)  # INCREASED: 64x64 is too small for video features
EPOCHS = 50 

print(f"Batch: {BATCH_SIZE} | Image: {IMG_SIZE} | Epochs: {EPOCHS}\n")

# ============================================================================
# WEBCAM SIMULATOR - THE "DIRTY DATA" AUGMENTATION
# ============================================================================
# This trains the model to handle bad lighting and off-center hands

augmentation = tf.keras.Sequential([
    # 1. Geometry (Simulate imperfect hand tracking)
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.15),          # Reduced rotation (hands don't spin 360)
    tf.keras.layers.RandomZoom(0.2),               # Increased zoom (hand moving back/forth)
    tf.keras.layers.RandomTranslation(0.1, 0.1),   # Hand isn't always perfectly centered
    
    # 2. Lighting & Quality (CRITICAL for webcam video)
    tf.keras.layers.RandomContrast(0.2),           # Webcams often have bad contrast
    tf.keras.layers.RandomBrightness(0.2),         # Simulates dark rooms or bright windows
], name="webcam_augmentation")

# ============================================================================
# LOAD DATA - STREAMING ONLY
# ============================================================================

print("Loading data (streaming mode)...")

# NOTE: Make sure TRAIN_PATH and TEST_PATH are defined in your environment!
# If you are copy-pasting this, ensure those variables exist.

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH,
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"✅ {num_classes} classes")

# ============================================================================
# PIPELINE SETUP
# ============================================================================

normalization = tf.keras.layers.Rescaling(1./255)

def prepare(ds, shuffle=False, augment=False):
    if shuffle:
        ds = ds.shuffle(1000)
    
    # Normalize FIRST
    ds = ds.map(lambda x, y: (normalization(x), y))
    
    # Augment SECOND (using the new Webcam Simulator)
    if augment:
        ds = ds.map(lambda x, y: (augmentation(x, training=True), y))
    
    # Prefetch for speed
    return ds.prefetch(tf.data.AUTOTUNE)

train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

print("✅ Pipeline ready\n")

# ============================================================================
# TEST PIPELINE SPEED
# ============================================================================

print("Testing pipeline speed...")
start = time.time()
for i, (images, labels) in enumerate(train_ds.take(20)):
    if i % 5 == 0:
        print(f"  Batch {i+1}/20... {time.time()-start:.1f}s")
elapsed = time.time() - start
print(f"✅ 20 batches in {elapsed:.1f}s ({20/elapsed:.1f} batch/sec)")

if elapsed > 40: # Increased threshold slightly for bigger images
    print("⚠️ WARNING: Pipeline is SLOW. Reduce batch_size or restart kernel.")
else:
    print("✅ Pipeline speed OK!\n")

# ============================================================================
# MODEL - UPDATED FOR NEW RESOLUTION
# ============================================================================


model = Sequential([
    # Block 1
    # Using IMG_SIZE variable so it updates automatically
    Conv2D(32, 3, activation='relu', padding='same', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    BatchNormalization(),
    Conv2D(32, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.25),
    
    # Block 2
    Conv2D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.25),
    
    # Block 3
    Conv2D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.3),
    
    # Classifier
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

print(f"Model: {model.count_params():,} params")
print(f"Size: {model.count_params()*4/1024/1024:.1f} MB\n")

# ============================================================================
# COMPILE & CALLBACKS
# ============================================================================

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

class ProgressTracker(tf.keras.callbacks.Callback):
    def __init__(self):
        self.start_time = time.time()
        self.epoch_start = None
    
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()
        elapsed = (time.time() - self.start_time) / 3600
        remaining = 8 - elapsed
        print(f"\n⏰ Epoch {epoch+1}/{EPOCHS} | Time used: {elapsed:.1f}h | Remaining: {remaining:.1f}h")
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_time = (time.time() - self.epoch_start) / 60
        print(f"✅ Epoch took {epoch_time:.1f} min | Val Acc: {logs['val_accuracy']*100:.2f}%")

callbacks = [
    ProgressTracker(),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
    ModelCheckpoint('best_asl_video_robust.keras', monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)
]

# ============================================================================
# TRAIN
# ============================================================================

print("\n" + "="*60)
print("STARTING ROBUST TRAINING")
print("="*60)

try:
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=2 
    )
    
    print("\n✅ TRAINING COMPLETED!")
    
except tf.errors.ResourceExhaustedError as e:
    print("\n❌ OUT OF MEMORY!")
    print("EMERGENCY FIX:")
    print("1. Restart Runtime")
    print("2. Change BATCH_SIZE = 4")
    raise

except Exception as e:
    print(f"\n❌ ERROR: {e}")
    raise

# ============================================================================
# RESULTS & SAVE
# ============================================================================

print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)

# Evaluate
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"Test Accuracy:  {test_acc*100:.2f}%")

# Save
model.save("asl_video_robust.keras")
print("\n✅ Model saved: asl_video_robust.keras")

# TFLite conversion
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("asl_video_robust.tflite", "wb") as f:
    f.write(tflite_model)

print(f"✅ TFLite saved: {len(tflite_model)/1024:.1f} KB")

In [ ]:
# ============================================================================
# 1. SAFER AUGMENTATION (Toned Down)
# ============================================================================
augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),    # Reduced from 0.15
    tf.keras.layers.RandomZoom(0.1),        # Reduced from 0.2 (Less risk of cropping fingers)
    # REMOVED RandomTranslation (It was likely pushing hands off-screen)
    
    # Lighting is still important, but let's be gentler
    tf.keras.layers.RandomContrast(0.1),
    tf.keras.layers.RandomBrightness(0.1),
], name="webcam_augmentation")

# ============================================================================
# 2. STRONGER MODEL (Lower Dropout = Easier to Learn)
# ============================================================================
model = Sequential([
    # Block 1
    Conv2D(32, 3, activation='relu', padding='same', input_shape=(96, 96, 3)),
    BatchNormalization(),
    MaxPooling2D(2),
    # REMOVED Dropout here (Let early layers learn features fully)
    
    # Block 2
    Conv2D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.2), # Reduced from 0.25
    
    # Block 3
    Conv2D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.2), # Reduced from 0.3
    
    # Classifier
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3), # Reduced from 0.5 (Critical change!)
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0005), # Slightly lower LR for stability
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
class ProgressTracker(tf.keras.callbacks.Callback):
    def __init__(self):
        self.start_time = time.time()
        self.epoch_start = None
    
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()
        elapsed = (time.time() - self.start_time) / 3600
        remaining = 8 - elapsed
        print(f"\n⏰ Epoch {epoch+1}/{EPOCHS} | Time used: {elapsed:.1f}h | Remaining: {remaining:.1f}h")
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_time = (time.time() - self.epoch_start) / 60
        print(f"✅ Epoch took {epoch_time:.1f} min | Val Acc: {logs['val_accuracy']*100:.2f}%")

callbacks = [
    ProgressTracker(),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
    ModelCheckpoint('best_asl_video_robust.keras', monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)
]


In [ ]:
# ============================================================================
# TRAIN
# ============================================================================

print("\n" + "="*60)
print("STARTING ROBUST TRAINING")
print("="*60)

try:
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=2 
    )
    
    print("\n✅ TRAINING COMPLETED!")
    
except tf.errors.ResourceExhaustedError as e:
    print("\n❌ OUT OF MEMORY!")
    print("EMERGENCY FIX:")
    print("1. Restart Runtime")
    print("2. Change BATCH_SIZE = 4")
    raise

except Exception as e:
    print(f"\n❌ ERROR: {e}")
    raise

# ============================================================================
# RESULTS & SAVE
# ============================================================================

print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)

# Evaluate
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"Test Accuracy:  {test_acc*100:.2f}%")

# Save
model.save("asl_video_robust.keras")
print("\n✅ Model saved: asl_video_robust.keras")

# TFLite conversion
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("asl_video_robust.tflite", "wb") as f:
    f.write(tflite_model)

print(f"✅ TFLite saved: {len(tflite_model)/1024:.1f} KB")

In [ ]:
model.save("asl_tiny_model_4.keras")


In [ ]:
import shutil

shutil.make_archive("asl_tiny_savedmodel", 'zip', "asl_tiny_savedmodel")


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os


2025-12-17 06:32:33.533166: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765953154.102510      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765953154.246593      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [33]:
TRAIN_DIR="/kaggle/input/asl-dataset-mine/archive/ASL_Alphabet_Dataset/asl_alphabet_train/"

TEST_DIR="/kaggle/input/asl-dataset-mine/archive/ASL_Alphabet_Dataset/asl_alphabet_test/"

In [25]:
IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 30
NUM_CLASSES = 29    

In [26]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)


In [27]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 223074 images belonging to 29 classes.
Found 28 images belonging to 28 classes.


In [28]:
weight_decay = 1e-4

model = models.Sequential([
    layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),

    layers.Conv2D(32, (3,3), activation='relu',
                  kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu',
                  kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), activation='relu',
                  kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),

    layers.Conv2D(256, (3,3), activation='relu',
                  kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.4),

    layers.Flatten(),

    layers.Dense(256, activation='relu',
                 kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    layers.Dense(NUM_CLASSES, activation='softmax')
])


In [29]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 62, 62, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 29, 29, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 4, 4, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 4, 4, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 29)             │         7,453 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 661,213 (2.52 MB)

 Trainable params: 659,741 (2.52 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [30]:
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6
    )
]


In [31]:
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=callbacks
)


Epoch 1/30
 382/6972 ━━━━━━━━━━━━━━━━━━━━ 28:45 262ms/step - accuracy: 0.0544 - loss: 4.3454

KeyboardInterrupt: 

In [ ]:
model.save("asl_cnn_model_5.h5")


In [39]:
TRAIN_PATH="/kaggle/input/asl-updated-data/archive(1)/archive/ASL_Alphabet_Dataset/asl_alphabet_train/"
TEST_PATH="/kaggle/input/asl-updated-data/archive(1)/archive/ASL_Alphabet_Dataset/asl_alphabet_test/"

In [40]:
len(os.listdir(TEST_PATH))

28

In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import os

# --- Configuration ---
# DATASET_DIR = "/kaggle/input/asl-alphabet/asl_alphabet_train"
IMG_SIZE = (64, 64)
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 29
LEARNING_RATE = 0.001

# --- Data Preparation ---

# Data Augmentation Configuration
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

print(f"Loading data from {DATASET_DIR}...")

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# validation_generator = test_datagen.flow_from_directory(
#     TEST_DIR,
#     target_size=IMG_SIZE,
#     batch_size=BATCH_SIZE,
#     class_mode='categorical',
#     subset='validation',
#     shuffle=False
# )

# --- Model Architecture ---
def build_model():
    model = Sequential()

    # First Block
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', 
                     input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
                     kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2)) # Light dropout after conv block

    # Second Block
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # Third Block
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))
    
    # Fourth Block
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # Dense Layers
    model.add(Flatten())
    
    model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5)) # Configurable high dropout

    model.add(Dense(NUM_CLASSES, activation='softmax'))

    return model

model = build_model()

# --- Compilation ---
optimizer = Adam(learning_rate=LEARNING_RATE)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# --- Training ---
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

# --- Save Model ---
model.save('asl_model_5.h5')
print("Model training complete and saved as asl_model.h5")


SyntaxError: '(' was never closed (2994836406.py, line 20)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import os
import sys

# --- Configuration ---
DATASET_DIR = "/kaggle/input/asl-alphabet/asl_alphabet_train"
IMG_SIZE = (64, 64)
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 29
LEARNING_RATE = 0.001

# --- Data Preparation ---
if not os.path.exists(DATASET_DIR):
    print(f"ERROR: Dataset directory not found at {DATASET_DIR}")
    print("Please check your dataset path.")
    sys.exit(1)


# Data Augmentation Configuration
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

print(f"Loading data from {DATASET_DIR}...")

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Safety check for empty generators
if train_generator.samples == 0:
    print("ERROR: Training set is empty. Check dataset structure.")
    sys.exit(1)
if validation_generator.samples == 0:
    print("ERROR: Validation set is empty. Check dataset structure or validation_split.")
    sys.exit(1)

# --- Model Architecture ---
def build_model():
    model = Sequential()

    # First Block
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', 
                     input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
                     kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2)) # Light dropout after conv block

    # Second Block
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # Third Block
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))
    
    # Fourth Block
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # Dense Layers
    model.add(Flatten())
    
    model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5)) # Configurable high dropout

    model.add(Dense(NUM_CLASSES, activation='softmax'))

    return model

model = build_model()

# --- Compilation ---
optimizer = Adam(learning_rate=LEARNING_RATE)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# --- Training ---
# Using len(generator) automatically calculates the correct number of steps (samples // batch_size + 1 if remainder)
# This prevents crashes when samples < batch_size or when integer division results in 0
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=EPOCHS
)

# --- Save Model ---
model.save('asl_model_5.h5')
print("Model training complete and saved as asl_model.h5")


Loading data from /kaggle/input/asl-alphabet/asl_alphabet_train...
Found 178472 images belonging to 29 classes.
Found 44602 images belonging to 29 classes.


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)              │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_31          │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_34          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 29)             │        14,877 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,504,925 (9.56 MB)

 Trainable params: 2,502,941 (9.55 MB)

 Non-trainable params: 1,984 (7.75 KB)

Epoch 1/20
   5/5578 ━━━━━━━━━━━━━━━━━━━━ 7:45 84ms/step - accuracy: 0.0012 - loss: 6.5650     